In [2]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import requests
import sqlite3 as sql
import pandas as pd
import numpy as np
import networkx as ntx 
import time 
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from config_new import config
import pytrends
import json

from nlp_helper import *
base_urls = {'NRSC':'https://www.nrsc.org/press-releases/',
'DSCC':'https://www.dscc.org/news/',
'DCCC':'https://dccc.org/press-release/',
'NRCC':'https://www.nrcc.org/press-room/',
'RNC' :'https://www.gop.com/press-release/',
'DNC':'https://democrats.org/news/'}

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-d

In [3]:
# con = sql.connect(config['database'])
# cur = con.cursor()
# cur.execute(""" DROP TABLE IF EXISTS vocab_urls; """)
# cur.execute("""DROP TABLE IF EXISTS urls_text; """)
# cur.execute("""DROP TABLE IF EXISTS static_text; """)
# cur.execute("""DROP TABLE IF EXISTS text_vader_cos_similarity; """)
# cur.execute("""DROP TABLE IF EXISTS title_vader_cos_similarity; """)
# cur.execute("""DROP TABLE IF EXISTS base_url; """)
# cur.execute("""DROP TABLE IF EXISTS catagory; """)               
# cur.execute("""CREATE TABLE IF NOT EXISTS catagory (id INTEGER PRIMARY KEY AUTOINCREMENT, catagory TEXT)
# """)
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS base_url (id INTEGER PRIMARY KEY AUTOINCREMENT, catagory_id_array TEXT, scheme TEXT, netloc TEXT, path TEXT, slug TEXT )
# """)
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS vocab_urls (id INTEGER PRIMARY KEY AUTOINCREMENT, base_url_id INTEGER, path TEXT)
# """)
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS urls_text (id INTEGER PRIMARY KEY AUTOINCREMENT, vocab_url_id INTEGER, p_tag_list TEXT, a_tag_list BLOB, title_tag_text TEXT, pull_dms TEXT)
# """)
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS static_text (id INTEGER PRIMARY KEY AUTOINCREMENT, base_url_id INTEGER,  text_block TEXT, type TEXT)
# """)
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS text_vader_cos_similarity (id INTEGER PRIMARY KEY AUTOINCREMENT, urls_text_id INTEGER,bag_a_words_json BLOB, Tfidf_json BLOB, sediment_neg REAL, sediment_neu REAL, sediment_pos REAL, sediment_comp REAL, week_average_similarity REAL )
# """)
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS title_vader_cos_similarity (id INTEGER PRIMARY KEY AUTOINCREMENT, urls_text_id INTEGER, sediment_neg REAL, sediment_neu REAL, sediment_pos REAL, sediment_comp REAL, post_vocab_similarity REAL )
# """)
# cur.execute("""CREATE VIEW IF NOT EXISTS Report 
#                 AS 
#                 SELECT slug, a.netloc || CASE WHEN a.path = '/' THEN '' ELSE a.path END || b.path AS 'url',
#                     c.p_tag_list,
#                     c.title_tag_text,
#                     d.sediment_comp AS title_comp,
# 					d.post_vocab_similarity AS title_comp_post,
#                     e.bag_a_words_json,
# 					e.Tfidf_json,
# 					e.sediment_comp AS post_comp
#                 FROM base_url a
#                 JOIN vocab_urls b ON a.id = b.base_url_id
#                 JOIN urls_text c ON b.id = c.vocab_url_id
#                 JOIN title_vader_cos_similarity d ON c.id = d.urls_text_id
# 				JOIN text_vader_cos_similarity e ON c.id = e.urls_text_id""")
# cur.execute(f"INSERT INTO catagory (catagory) VALUES ('Republican'),('Democrate'),('3rd Party'),('Natioanl'),('Local'),('Party'),('Independent'),('News')")
# for key, value in base_urls.items():
#     u = urlparse(value)
#     cur.execute(f"INSERT INTO base_url (slug, scheme, netloc, path) VALUES ('{key}','{u.scheme}','{u.netloc}','{u.path}')")
# con.commit()
# con.close()

In [3]:
def removeValue(givenlist, value):
    while value in givenlist:
        givenlist.remove(value)
    return givenlist

def link_gather(base_urls_idx,con):
    cur = con.cursor()
    base_urls = pd.read_sql_query(f"SELECT id, a.scheme || '://' || a.netloc || a.path as baseURL from base_url a where id = {base_urls_idx}", con)
    base_urls = base_urls['baseURL'].to_list()[0]
    x = requests.get(base_urls, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(x.text, 'html.parser')
    el = soup.body.find_all('a', href=True)
    urls = [i['href'] for i in el]
    post = []
    for i in urls:
        if (urlparse(base_urls).path != urlparse(i).path and urlparse(base_urls).netloc == urlparse(i).netloc) or urlparse('/news/breaking-news/').netloc == '':
            post.append(i)
    post = list(set(post))
    df_post = pd.read_sql_query(f"""
        SELECT b.scheme || '://' || b.netloc || a.path AS url_path FROM vocab_urls a JOIN base_url b ON b.id = a.base_url_id WHERE base_url_id = {base_urls_idx}
    """, con)
    scrapped = df_post['url_path'].to_list()
    for link in scrapped:
        post = removeValue(post,link)
    for link in post:
        u = urlparse(link)
        cur.execute(f"INSERT INTO vocab_urls (base_url_id, path) VALUES ({base_urls_idx}, '{u.path}')")
        con.commit()
    return post

def static(current,previous):
    static_text = []
    for i in current:
        for j in previous:
            try:
                score = np.round(np.sum(cos_similarity([i,j])[0]))
                if score == 2 :
                    static_text.append(i)
            except:
                static_text.append(i)
    static_text = list(set(static_text))
    return static_text

def blog_post_scrap(vurl_idx,url,con):
    cur = con.cursor()
    diff_clean = {'current':{'title':None,'links':None,'text':None}, 'previous':{'title':None,'links':None,'text':None}}
    try:
        link = url
        vurl_idx = vurl_idx
        x = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(x.text, 'html.parser')
        p = soup.body.find_all('p')
        a = soup.body.find_all('a', href=True)
        title = soup.find('title').text
        links = [i['href'] for i in a]
        text = []
        for i in p:
            try:
                j = i.text.replace(" ", "").replace('\n',' ')
                if len(j)>0:
                    text.append(i.text.replace('\n',' '))
            except:
                continue
        static_df = pd.read_sql_query(f"""SELECT text_block, type FROM static_text WHERE  base_url_id = {vurl_idx}""",con)
        if not static_df['text_block'].to_list():
            if not diff_clean['current']['text']:
                diff_clean['current']['url'] = link
                diff_clean['current']['idx'] = vurl_idx
                diff_clean['current']['text'] = text
                diff_clean['current']['links'] = links
                diff_clean['current']['title'] = title
            else:
                diff_clean['previous']['url'] = link
                diff_clean['previous']['idx'] = vurl_idx
                diff_clean['previous']['text'] = text
                diff_clean['previous']['links'] = links
                diff_clean['previous']['title'] = title
            if  diff_clean['current']['text'] and diff_clean['previous']['text']:
                current_text = diff_clean['current']['text']
                previous_text = diff_clean['previous']['text']
                static_text = static(current_text,previous_text)
                current_link = diff_clean['current']['links']
                previous_link = diff_clean['previous']['links']
                static_links = static(current_link,previous_link)
                for s_text in static_text:
                    cur.execute(f"""INSERT INTO static_text (base_url_id, text_block, type) VALUES ({vurl_idx} ,'{s_text.replace("'","''").replace('"','""')}', 'text') """)
                    con.commit()
                for a_text in static_links:
                    cur.execute(f"""INSERT INTO static_text (base_url_id, text_block, type) VALUES ({vurl_idx} ,'{a_text}', 'link') """)
                    con.commit()
                for key in diff_clean.keys():
                    blog_post_scrap(diff_clean[key]['idx'],diff_clean[key]['url'],con)
        else:
            for idx, row in static_df.iterrows():
                if row['type'] == 'text':
                    text = removeValue(text, row['text_block'])
                elif row['type'] == 'link':
                    links = removeValue(links, row['text_block'])
            text = ' '.join(text)
            text = text.replace("'","''")
            links = ','.join(links)
            cur.execute(f"""INSERT INTO urls_text (vocab_url_id , p_tag_list , a_tag_list , title_tag_text , pull_dms) VALUES ({vurl_idx},'{text.replace("'","''")}','{links}','{title.replace("'","''")}','{time.strftime('%Y-%m-%d', time.localtime())}')""")
            con.commit()
    except Exception as e:
        print(e)

In [4]:
## Dummy Inputs
con = sql.connect(config['database'])
links = link_gather(idx_dummy,con)
cur = con.cursor()
post_data = pd.read_sql_query(f"""SELECT a.id, b.scheme || '://' || b.netloc || a.path AS url FROM vocab_urls a JOIN base_url b ON b.id = a.base_url_id WHERE a.base_url_id = {idx_dummy} and a.id NOT IN (SELECT vocab_url_id FROM urls_text)""", con)
# Function Blog Posts
for idx, row in post_data.iterrows():
    blog_post_scrap(row['id'],row['url'],con)
con.close()
# # Text NLP
con = sql.connect(config['database'])
id_list = pd.read_sql_query("SELECT id FROM urls_text WHERE id NOT IN (SELECT urls_text_id FROM title_vader_cos_similarity) ",con)
id_list = id_list['id'].tolist()
for idnum in id_list:
    try:
        row = pd.read_sql_query(f"SELECT * FROM urls_text WHERE id = {idnum} ",con)
        idx = row['id'].values[0]

        title_tag_text = row['title_tag_text'].values[0]
        title_tag_block = processed_feature(title_tag_text)
        title_vadar = vadar(title_tag_block)

        p_tag_list = re.sub(r'[\d.]*\d+', '',row['p_tag_list'].values[0])
        p_tag_block = processed_feature(p_tag_list)
        p_vocab = create_bag_of_words(p_tag_block)
        p_vadar = vadar(p_tag_block)
        p_Tfidf = Tfidf(p_tag_list.split('.'))
        dict_Tfidf = {key:value for key, value in p_Tfidf}
        dict_Tfidf = dict(sorted(dict_Tfidf.items(),key = lambda x:x[1], reverse = True))
        avg = sum(dict_Tfidf.values())/len(dict_Tfidf.keys())
        dict_Tfidf_insert = {}
        for key, value in dict_Tfidf.items():
            if value > avg:
                dict_Tfidf_insert[key] =value 
        
        dict_vocab_insert = dict(nltk.FreqDist(p_vocab).most_common(len(dict_Tfidf_insert.keys())))
        tvp = cos_similarity([title_tag_block,' '.join(list(dict_vocab_insert.keys()))])[0][1]

        cur.execute(f"""INSERT INTO title_vader_cos_similarity (urls_text_id, sediment_neg, sediment_neu, sediment_pos, sediment_comp, post_vocab_similarity ) VALUES ({idnum}, {title_vadar['neg']}, {title_vadar['neu']}, {title_vadar['pos']}, {title_vadar['comp']}, {tvp});""")
        cur.execute(f"""INSERT INTO text_vader_cos_similarity (urls_text_id, bag_a_words_json, Tfidf_json, sediment_neg, sediment_neu, sediment_pos, sediment_comp) VALUES ({idnum}, '{json.dumps(dict_vocab_insert)}', '{json.dumps(dict_Tfidf)}', {p_vadar['neg']}, {p_vadar['neu']}, {p_vadar['pos']}, {p_vadar['comp']});""")
        con.commit()
    except Exception as e:
        continue
con.close()

Failed to parse: https://bonnercountydailybee.comtel:2082639534
